# Data cleaning - Basics

This notebook corresponds to mission 17 of [dataquest](https://www.dataquest.io).

In [1]:
import numpy as np
import pandas as pd

**Encoding while importing data**
<br>
<br>_Most used:_
* Default: encoding = "UTF-8"
* "Latin-1" (also known as ISO-8895-1)
* "Windows-1251"

In [2]:
laptops= pd.read_csv("laptops.csv", encoding="Latin-1")
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


In [3]:
#In this case UTF-8 will return a error
#laptops1 = pd.read_csv("laptops.csv", encoding="UTF-8") #Or pd.read_csv("laptops.csv")
#laptops1.info()

In [4]:
laptops2 = pd.read_csv("laptops.csv", encoding="Windows-1251")
laptops2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
Manufacturer                1303 non-null object
Model Name                  1303 non-null object
Category                    1303 non-null object
Screen Size                 1303 non-null object
Screen                      1303 non-null object
CPU                         1303 non-null object
RAM                         1303 non-null object
 Storage                    1303 non-null object
GPU                         1303 non-null object
Operating System            1303 non-null object
Operating System Version    1133 non-null object
Weight                      1303 non-null object
Price (Euros)               1303 non-null object
dtypes: object(13)
memory usage: 132.4+ KB


**It can be seen that our file has loaded well in all encodes above, but there will be times that the file won't open correctly without its correct encode**

## Cleaning column names

In [5]:
laptops.columns

Index(['Manufacturer', 'Model Name', 'Category', 'Screen Size', 'Screen',
       'CPU', 'RAM', ' Storage', 'GPU', 'Operating System',
       'Operating System Version', 'Weight', 'Price (Euros)'],
      dtype='object')

* Removing any whitespace from the start and end of the labels - if you look closely you'll notice that the " Storage" column label has a space in front of it. These quirks with column labels are often hard to spot, so removing them at the start will save you pain in the long run.
* Replacing spaces with underscores and remove special characters - this will make things consistent, and also allows for the use of dot accessors for those who prefer that.
* Make all labels lowercase - this is good for consistency, and means you'll never have to remember what is capitalized and how.
Shorten any long column names - this helps to keep your code easier to read, especially when you are using method chaining.

In [6]:
#Creating a method to clean the names
def name_cleaning(col):
    col = col.replace("Operating System", "os")
    col = col.strip() #Cleans space at begining and the end of the string
    col = col.replace(" ", "_")
    col = col.replace("(", "")
    col = col.replace(")", "")
    col = col.lower()
    return col

#Creating a list to receive the cleaned names
new_col_names = []

#looping through the column names
for col_name in laptops.columns:
    new_col_names.append(name_cleaning(col_name))
    
#Passing the values
laptops.columns = new_col_names
laptops.columns

Index(['manufacturer', 'model_name', 'category', 'screen_size', 'screen',
       'cpu', 'ram', 'storage', 'gpu', 'os', 'os_version', 'weight',
       'price_euros'],
      dtype='object')

---

## Converting text to numeric data

Whenever we're converting text to numeric data, we can follow this data cleaning workflow:

<img src="cleaning_workflow.svg" style="height:250px;float:left;" >

**Example Screen_size**:

1. Explore data in column:

In [7]:
laptops["screen_size"].unique()

array(['13.3"', '15.6"', '15.4"', '14.0"', '12.0"', '11.6"', '17.3"',
       '10.1"', '13.5"', '12.5"', '13.0"', '18.4"', '13.9"', '12.3"',
       '17.0"', '15.0"', '14.1"', '11.3"'], dtype=object)

2. Identify patterns and special cases:

As we can see on the code above [unique](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html) is realy useful when it comes to first explore a column, we can see that there is no null values in the column and all values are numeric with quotation marks.

3. Remove non-digit characters

In [8]:
#For our analyses the quotation marks wont be necessary, and working with numeric values for screen size will help us.
laptops["screen_size"] = laptops["screen_size"].str.replace('"','')

print(laptops["screen_size"].unique())

['13.3' '15.6' '15.4' '14.0' '12.0' '11.6' '17.3' '10.1' '13.5' '12.5'
 '13.0' '18.4' '13.9' '12.3' '17.0' '15.0' '14.1' '11.3']


**Obs:** In the code above in order to avoid unnecessary loops,  has been used [series.str.replace](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.replace.html) instead of just .replace(), this is really important because we are working with a vector (pd.series), and the serires.str.replace is a vectorized str.replace(). 

4. Convert the column to a numeric type

In [9]:
#Float will be used, since some values have decimals
laptops["screen_size"] = laptops["screen_size"].astype(float)

print(laptops["screen_size"].dtype)

float64


5. Rename column (if required)

In [10]:
laptops.rename({"screen_size": "screen_size_inches"}, axis=1, inplace=True)
print(laptops.dtypes)

manufacturer           object
model_name             object
category               object
screen_size_inches    float64
screen                 object
cpu                    object
ram                    object
storage                object
gpu                    object
os                     object
os_version             object
weight                 object
price_euros            object
dtype: object


### Examples:
The same as above, but for the column 'ram'

In [11]:
laptops['ram'].unique()

array(['8GB', '16GB', '4GB', '2GB', '12GB', '6GB', '32GB', '24GB', '64GB'],
      dtype=object)

In [12]:
laptops['ram'] = laptops['ram'].str.replace('GB', '').astype(int)
laptops.rename({'ram':'ram_gb'}, inplace=True, axis=1)
laptops.dtypes

manufacturer           object
model_name             object
category               object
screen_size_inches    float64
screen                 object
cpu                    object
ram_gb                  int32
storage                object
gpu                    object
os                     object
os_version             object
weight                 object
price_euros            object
dtype: object

There is other problems that may appear like in the weight column, that the text with the numeric may vary ('kg' or 'kgs').
<br> Other problems are dot/comma with numerical values, python will use dot, so whenever numeric values use comma, like in the price column it must be changed.

In [13]:
print("Some weights: ", list(laptops.loc[:3,"weight"]))
print("Some prices: ", list(laptops.loc[:3,"price_euros"]))

Some weights:  ['1.37kg', '1.34kg', '1.86kg', '1.83kg']
Some prices:  ['1339,69', '898,94', '575,00', '2537,45']


In [14]:
laptops["weight"] = (laptops["weight"]
                        .str.replace("kgs", "")
                        .str.replace("kg", "")
                        .astype(float)
                    )
laptops["price_euros"] = (laptops["price_euros"]
                        .str.replace(",", ".")
                        .astype(float)
                    )
laptops.rename({"weight":"weight_kg"}, axis=1, inplace=True)

print("Some weights: ", list(laptops.loc[:3,"weight_kg"]))
print("Some prices: ", list(laptops.loc[:3,"price_euros"]))

Some weights:  [1.37, 1.34, 1.86, 1.83]
Some prices:  [1339.69, 898.94, 575.0, 2537.45]


**Always important to rembember [series.describe()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.describe.html) that give us the basic statistics of a serie:**

In [15]:
laptops["weight_kg"].describe()

count    1303.000000
mean        2.038734
std         0.665475
min         0.690000
25%         1.500000
50%         2.040000
75%         2.300000
max         4.700000
Name: weight_kg, dtype: float64

<br>

### Getting a part of the text

First lets take a look at the columns gpu and cpu:

In [16]:
print("GPU:")
print(laptops["cpu"].head(3))
print("\nCPU:")
print(laptops["gpu"].head(3))

GPU:
0          Intel Core i5 2.3GHz
1          Intel Core i5 1.8GHz
2    Intel Core i5 7200U 2.5GHz
Name: cpu, dtype: object

CPU:
0    Intel Iris Plus Graphics 640
1          Intel HD Graphics 6000
2           Intel HD Graphics 620
Name: gpu, dtype: object


As it can be seen above, in both columns the data starts with the manufacturer as the first word. <br>
So we will try to create columns of just the manufacturer, for this we will use [Series.str.split()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html), wich is a vectorize split. Split creates a list by separating words as elements by spaces on the string. 

In [17]:
laptops["gpu_manufacturer"] = (laptops["gpu"]
                                    .str.split(n=1,expand=True) 
                                    .iloc[:,0]
                               )
laptops["cpu_manufacturer"] = (laptops["cpu"]
                                    .str.split(n=1, expand=True)
                                    .iloc[:,0]
                              )

**Obs:**
* **n** is a parameter that you can limit how many splits you want. By passing 1, you divide the string in a list of two elements, the first word and the rest of the string.
* **expand** when is set as TRUE, the series split creates a DataFrame into a DataFrame, in other words, expand the splitted strings into separate columns.

<img src="str_split_4.svg" style="height:150px;float:left;">

<br>

### Getting text on other positions

Now lets take a look ate the screen column:

In [18]:
laptops["screen"].unique()[:10]

array(['IPS Panel Retina Display 2560x1600', '1440x900',
       'Full HD 1920x1080', 'IPS Panel Retina Display 2880x1800',
       '1366x768', 'IPS Panel Full HD 1920x1080',
       'IPS Panel Retina Display 2304x1440',
       'IPS Panel Full HD / Touchscreen 1920x1080',
       'Full HD / Touchscreen 1920x1080',
       'Touchscreen / Quad HD+ 3200x1800'], dtype=object)

We can see that the resolution is in the last position, but if we use split would problem, because it will start by the left and then will mess up the reolustions positions as we can see bellow:

In [19]:
print(laptops.loc[:4, "screen"].str.split(expand=True))

          0      1          2        3          4
0       IPS  Panel     Retina  Display  2560x1600
1  1440x900   None       None     None       None
2      Full     HD  1920x1080     None       None
3       IPS  Panel     Retina  Display  2880x1800
4       IPS  Panel     Retina  Display  2560x1600


For solving this problem will be used [series.str.rsplit()](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.rsplit.html), which splits from the end:

In [20]:
print(laptops.loc[:4, "screen"].str.rsplit(n=1, expand=True))

                          0          1
0  IPS Panel Retina Display  2560x1600
1                  1440x900       None
2                   Full HD  1920x1080
3  IPS Panel Retina Display  2880x1800
4  IPS Panel Retina Display  2560x1600


This last result is much closer from what we are looking for.<br>
The problem now resumes to some lines where the right column is None, but always in those places the value on the left column is the resolution, so it just some treatment:

In [21]:
df_res = laptops["screen"].str.rsplit(n=1, expand=True)
df_res.columns = ["A", "B"] # giving the columns string labels makes them easier to work with
df_res.loc[df_res["B"].isnull(), "B"] = df_res["A"]  #Where the column B is null, pass the A value to it
print(df_res.iloc[:10]) #Confirming 

                          A          B
0  IPS Panel Retina Display  2560x1600
1                  1440x900   1440x900
2                   Full HD  1920x1080
3  IPS Panel Retina Display  2880x1800
4  IPS Panel Retina Display  2560x1600
5                  1366x768   1366x768
6  IPS Panel Retina Display  2880x1800
7                  1440x900   1440x900
8                   Full HD  1920x1080
9         IPS Panel Full HD  1920x1080


In [22]:
laptops["screen_resolution"] = df_res["B"]
print(laptops["screen_resolution"].unique())

['2560x1600' '1440x900' '1920x1080' '2880x1800' '1366x768' '2304x1440'
 '3200x1800' '1920x1200' '2256x1504' '3840x2160' '2160x1440' '2560x1440'
 '1600x900' '2736x1824' '2400x1600']


### Another Example:
Now lets extract the speed from cpu column, but this time we can extract as float:

In [23]:
laptops["cpu"].unique()[:5]

array(['Intel Core i5 2.3GHz', 'Intel Core i5 1.8GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Core i7 2.7GHz',
       'Intel Core i5 3.1GHz'], dtype=object)

In [24]:
#Creating a DataFrame separating the last word in a diffrent column from the rest of the string
df_cpu_speed = laptops["cpu"].str.rsplit(n=1, expand=True)
df_cpu_speed.columns = ["A", "B"]  # giving the columns string labels makes them easier to work with
df_cpu_speed["B"] = (df_cpu_speed["B"]
                                     .str.replace("GHz", "")
                                     .astype(float)
                    )
df_cpu_speed.loc[:3, "B"] #Confirming

0    2.3
1    1.8
2    2.5
3    2.7
Name: B, dtype: float64

In [25]:
laptops["cpu_speed_ghz"] = df_cpu_speed["B"] 
laptops["cpu_speed_ghz"].unique()

array([2.3 , 1.8 , 2.5 , 2.7 , 3.1 , 3.  , 2.2 , 1.6 , 2.  , 2.8 , 1.2 ,
       2.9 , 2.4 , 1.44, 1.5 , 1.9 , 1.1 , 1.3 , 2.6 , 3.6 , 3.2 , 1.  ,
       2.1 , 0.9 , 1.92])

---


## Inconsistent Values
**(really frequent issue)**<br>


One way it could be fixed is by using a boolean comparison and assignment (as we did before), but normally we may find multipple inconsisteces. Because of this the method which is most used is: the [Series.map()](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html) method. <br>
For this first we will work upon a simple example, a series with multiple errors:

In [26]:
fruit_array = np.array(["oranje", "bananna", "pair", "oranje", "oranje", "oranje"])
fruit_series = pd.Series(fruit_array)
fruit_series

0     oranje
1    bananna
2       pair
3     oranje
4     oranje
5     oranje
dtype: object

In [27]:
#First the map method requires a dictionary with the corrections
fruits_correction = {
                "oranje" : "orange",
                "pair" : "pear",
                "bananna" : "banana"
}

#Apply the method
fruit_series = fruit_series.map(fruits_correction)
fruit_series

0    orange
1    banana
2      pear
3    orange
4    orange
5    orange
dtype: object

#### There is a problem in pd.Series.map() method, wich is: if there ins't a value that is equal to one of the dictionary's keys, this value will be turned into None or NaN:

In [28]:
key_test = {
                "banana" : "Key_found"
}
fruit_series = fruit_series.map(key_test)
fruit_series

0          NaN
1    Key_found
2          NaN
3          NaN
4          NaN
5          NaN
dtype: object

<br>

**Applying to our Data:**

In [29]:
laptops["os"].value_counts()

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          13
Mac OS          8
Android         2
Name: os, dtype: int64

As it can be seen above, there is a repetion on the Apple's operating system:

In [30]:
mac_correction = {
                'Android': 'Android',
                'Chrome OS': 'Chrome OS',
                'Linux': 'Linux',
                'Mac OS': 'macOS',
                'No OS': 'No OS',
                'Windows': 'Windows',
                'macOS': 'macOS'
                }

laptops["os"] = laptops["os"].map(mac_correction)
laptops["os"].value_counts()

Windows      1125
No OS          66
Linux          62
Chrome OS      27
macOS          21
Android         2
Name: os, dtype: int64

---
## Treating Null

**Identifying:** There are two approaches we can use: the DataFrame.info() method and the DataFrame.isnull() method.

In [31]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 17 columns):
manufacturer          1303 non-null object
model_name            1303 non-null object
category              1303 non-null object
screen_size_inches    1303 non-null float64
screen                1303 non-null object
cpu                   1303 non-null object
ram_gb                1303 non-null int32
storage               1303 non-null object
gpu                   1303 non-null object
os                    1303 non-null object
os_version            1133 non-null object
weight_kg             1303 non-null float64
price_euros           1303 non-null float64
gpu_manufacturer      1303 non-null object
cpu_manufacturer      1303 non-null object
screen_resolution     1303 non-null object
cpu_speed_ghz         1303 non-null float64
dtypes: float64(4), int32(1), object(12)
memory usage: 168.0+ KB


There are two downsides to this approach - firstly the information is printed, so we can't easily work with it, and secondly looking at the number of non-null values can be harder to understand than looking at the number of null values. In contrast, DataFrame.isnull() returns a boolean dataframe with True and False indications for every value in the dataframe, and then we can use DataFrame.sum() to give us accounts– using a .sum() method on a boolean array will give us a count of the True values:

In [32]:
laptops.isnull().sum()

manufacturer            0
model_name              0
category                0
screen_size_inches      0
screen                  0
cpu                     0
ram_gb                  0
storage                 0
gpu                     0
os                      0
os_version            170
weight_kg               0
price_euros             0
gpu_manufacturer        0
cpu_manufacturer        0
screen_resolution       0
cpu_speed_ghz           0
dtype: int64

**Treating options:**
* Remove any rows that have missing values.
* Remove any columns that have missing values.
* Fill the missing values with some other value.
* Leave the missing values as is

The first two options, removing columns and/or rows with missing values is often used when preparing data for machine learning, as machine learning algorithms are unable to be trained on data that includes null values. The methods that we use to remove rows and columns with null values is the [DataFrame.dropna()](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method. As a result, removing columns and rows is commonly known as dropping.

**Obs:** Dropna deletes row or column wich has a null value on it.<br>
Choosing if it is a column or row, depends on the axis parameter pass to it: 
axis=1(Columns) axis=0(Rows).
<br>Remembering that axis default value is 0.

In [33]:
laptops_no_null_rows = laptops.dropna()
laptops_no_null_cols = laptops.dropna(axis=1)

<br>

### Treating without removing entire lines or columns

We are looking for treating the null values, but without removing whole lines or columns, because would may intefer directly on our analysis.
 One method is to explore all of the values in the column, for this we can use Series.value_counts() with the dropna=False parameter.  default, Series.value_counts() won't include null values in its output. This parameter allows us to explicitly indicate we want to see the null values:

In [34]:
print(laptops["os_version"].value_counts(dropna=False))

10      1072
NaN      170
7         45
X          8
10 S       8
Name: os_version, dtype: int64


**Now let see how are the OS's of the null values:**

In [35]:
os_with_null_values = laptops.loc[
                                laptops["os_version"].isnull(),
                                "os"
                                ]
print(os_with_null_values.value_counts())

No OS        66
Linux        62
Chrome OS    27
macOS        13
Android       2
Name: os, dtype: int64


Immediately we can observe a few things:
* Most of the missing values are actually when the laptop doesn't include any OS. This is an important distinction, because it's not so much that we don't know what the value is, as that there can't be a value.
* 13 of the laptops that come with macOS do not specify the version. Leaning on our knowledge of MacOS, we might know that the full name of macOS used to be Mac OS X, and so we might to fill these values to be more consistent.

In both of these cases, we can fill the missing values to make our data more correct. For the rest of the values, it's probably best to leave them as missing so we don't remove important values.<br>
First, let's explore those mac columns a bit more to make sure our intuition was correct:

In [36]:
mac_os_versions = laptops.loc[laptops["os"] == "macOS", "os_version"]

print(mac_os_versions.value_counts(dropna=False))

NaN    13
X       8
Name: os_version, dtype: int64


We were correct, all 8 rows that have the value X are macOS versions. We'll fill in all of the NaN values with X. We can use assignment with a boolean comparison to perform this replacement:

In [37]:
laptops.loc[laptops["os"] == "macOS", "os_version"] = "X"

For our other case, let's insert a No OS value into the os_version column for any laptop with a No OS value in the os column:

In [38]:
bool_list_no_os = laptops["os"] == "No OS"
laptops.loc[bool_list_no_os, "os_version"] = "Version Unknown"

In [39]:
value_counts_after = laptops.loc[laptops["os_version"].isnull(), "os"].value_counts()

---

Lets get the storage column and separate it in capacity with gb and type, and do it for 2 memories (max a laptop has on this dataset)

In [40]:
print(laptops.loc[70:90, 'storage'])

70               128GB SSD
71               256GB SSD
72               256GB SSD
73    128GB SSD +  1TB HDD
74                 1TB HDD
75                 1TB HDD
76                 2TB HDD
77    128GB SSD +  1TB HDD
78                 1TB HDD
79    128GB SSD +  1TB HDD
80               256GB SSD
81               512GB SSD
82               256GB SSD
83               128GB SSD
84                 1TB HDD
85    128GB SSD +  1TB HDD
86               256GB SSD
87               256GB SSD
88    128GB SSD +  1TB HDD
89               256GB SSD
90                 1TB HDD
Name: storage, dtype: object


In [41]:
print(laptops["storage"].unique())

['128GB SSD' '128GB Flash Storage' '256GB SSD' '512GB SSD' '500GB HDD'
 '256GB Flash Storage' '1TB HDD' '32GB Flash Storage'
 '128GB SSD +  1TB HDD' '256GB SSD +  256GB SSD' '64GB Flash Storage'
 '256GB SSD +  1TB HDD' '256GB SSD +  2TB HDD' '32GB SSD' '2TB HDD'
 '64GB SSD' '1TB Hybrid' '512GB SSD +  1TB HDD' '1TB SSD'
 '256GB SSD +  500GB HDD' '128GB SSD +  2TB HDD' '512GB SSD +  512GB SSD'
 '16GB SSD' '16GB Flash Storage' '512GB SSD +  256GB SSD'
 '512GB SSD +  2TB HDD' '64GB Flash Storage +  1TB HDD' '1GB SSD'
 '1TB HDD +  1TB HDD' '32GB HDD' '1TB SSD +  1TB HDD'
 '512GB Flash Storage' '128GB HDD' '240GB SSD' '8GB SSD' '508GB Hybrid'
 '512GB SSD +  1TB Hybrid' '256GB SSD +  1TB Hybrid']


In [42]:

def treating_storage(sto_str):
    size1 = 0
    type1 = None
    size2 = None
    type2 = None
    
    sto_list = sto_str.split()
    
    #Analyzing if there is a second storage 
    hd2 = False
    if("+" in sto_list):
            hd2 = True
        
    #Separating size 1
    if("TB" in sto_list[0]):
        sto_list[0] = sto_list[0].replace("TB","")
        size1 = int(sto_list[0]) * 1000
    else:
        sto_list[0] = sto_list[0].replace("GB","")
        size1 = int(sto_list[0])
        
    #Separating type 1
    if(sto_list[1] == "Flash"):
        type1 = "Flash Storage"
        
    else:
        type1 = sto_list[1]
        
    #Analysing second storage
    if(hd2 == False):
        return [size1, type1, size2, type2]
    else:
        #Locating + position on string list
        middle_string_position = sto_list.index("+")
        
        #Handling size2
        if("TB" in sto_list[middle_string_position + 1]):
            sto_list[middle_string_position + 1] = sto_list[middle_string_position + 1].replace("TB","")
            size2 = int(sto_list[middle_string_position + 1]) * 1000
        else:
            sto_list[middle_string_position + 1] = sto_list[middle_string_position + 1].replace("GB","")
            size2 = int(sto_list[middle_string_position + 1])
        
        #Handling type2
        type2 = sto_list[middle_string_position + 2]
        
        
        return [size1, type1, size2, type2]       
        
        